In [1]:
# Import libraries
from collections import Counter, OrderedDict
from itertools import chain
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
plt.rcParams["font.size"] = 15
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
import random
import seaborn as sns
from sklearn.manifold import TSNE
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Path to file
hotels_path = "../data/hotels_items.csv"
# Create dataframe
hotels_df = pd.read_csv(hotels_path, usecols = ["item_id", "properties", "city", "price"])

In [3]:
# Create item list
item_list = hotels_df["item_id"].tolist()

# Create indexes for each item
item_index = {item: idx for idx, item in enumerate(item_list)}
index_item = {idx: item for item, idx in item_index.items()}

# Create price list
price_list = hotels_df["price"].tolist()

# Create indexes for each item
price_index = {price: idx for idx, price in enumerate(price_list)}
index_price = {idx: price for price, idx in price_index.items()}

# Prepare properties column

# Split of pipe
hotels_df["properties"] = hotels_df["properties"].str.split("|")

# Convert all properties to lowercase
hotels_df["properties"] = hotels_df["properties"].apply(lambda x: [w.lower() for w in x])

# Create list
properties_list = hotels_df["properties"].tolist()

# Find set of unique properties and convert to a list
unique_properties = list(chain(*[list(set(tags)) for tags in properties_list]))
unique_properties = set(unique_properties)

# Create indexes for each property
property_index = {tag: idx for idx, tag in enumerate(unique_properties)}
index_properties = {idx: tag for tag, idx in property_index.items()}

# Prepare city column

# Convert all properties to lowercase
hotels_df["city"] = hotels_df["city"].apply(lambda x: x.lower())

# Create list
location_list = hotels_df["city"].tolist()

# Find set of unique locations and convert to a list
# unique_locations = list(chain(*[list(set(location)) for location in location_list]))
unique_locations = set(location_list)

# Create indexes for each property
location_index = {location: idx for idx, location in enumerate(unique_locations)}
index_location = {idx: location for location, idx in location_index.items()}

In [4]:
# Import embeddings model and weights
embeddings_model = models.load_model("../models/embeddings_third_attempt.h5")
embeddings_model.load_weights("../models/embeddings_third_attempt_weights.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
